In [1]:
import requests
from bs4 import BeautifulSoup
import nltk

In [2]:



url = 'https://www.yarsabazar.com/'  
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}



In [3]:
def ParseHTML(url,headers):
    response = requests.get(url,headers= headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        return soup
        
    else:
        print('Failed to retrieve the page')
        return -1
        

In [4]:
soup = ParseHTML(url,headers)

In [5]:
def SplitDocumentsByTags(soup):
    chunks = []

    for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6','a']):
        heading_text = heading.get_text()
        sibling = heading.find_next_sibling()
        content = []

        while sibling and sibling.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            content.append(sibling.get_text())
            sibling = sibling.find_next_sibling()

        chunks.append((heading_text, " ".join(content)))
    return chunks

In [6]:
SplitDocumentsByTags(soup)

[('', ''),
 ('', ''),
 ('Login', ''),
 ('Sign Up', ''),
 ('Search your desired ProductsVendorsCategories', ''),
 ('Industries', 'See all'),
 ('See all', ''),
 ('Agriculture, Livestock and Herbs',
  'Furniture, Wood and Metal Paper, Plastics and Packaging Clothing, Jewelry and Textile Food, Drinks and Grocery Medical, Pharma and Chemicals Electronics, IT and Networking Power, Energy and Electricity Tools, Machinery and Automobile Building, Construction and Paints'),
 ('Furniture, Wood and Metal',
  'Paper, Plastics and Packaging Clothing, Jewelry and Textile Food, Drinks and Grocery Medical, Pharma and Chemicals Electronics, IT and Networking Power, Energy and Electricity Tools, Machinery and Automobile Building, Construction and Paints'),
 ('Paper, Plastics and Packaging',
  'Clothing, Jewelry and Textile Food, Drinks and Grocery Medical, Pharma and Chemicals Electronics, IT and Networking Power, Energy and Electricity Tools, Machinery and Automobile Building, Construction and Paints')

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch

In [8]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [27]:
def GetEmbeddings(chunks):
    
    embeddings = []
    #print(chunks)
    for heading, text in chunks:
        #print(text)
        if text == "":
            continue
        inputs = tokenizer(text, return_tensors='pt', truncation=True)
        print(inputs)
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            embedding = torch.mean(last_hidden_states, dim=1)  # Mean pooling
            embeddings.append(embedding.squeeze().numpy())
    return embeddings


In [29]:

chunks = SplitDocumentsByTags(soup)
embeddings = GetEmbeddings(chunks)

(768,)